#Section 4.2: Sensitivity Analysis

This notebook shows how we can analyze sensitivity either through a brute force approach or by generating a sensitivity analysis report.

In [0]:
!pip install pulp
import pulp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Brute Force Sensitivity Tool

By changing the location of <code>a</code> in the LP, we can analyze sensitivity to different changes in the Lego LP.  We change the bounds or stepsize in <code>np.arange(-5,5,0.5)</code> to test different <code>a</code> values.

In [0]:
def run_pulp_model(a):
  model = pulp.LpProblem("Lego_Construction_Problem", pulp.LpMaximize)
  x1 = pulp.LpVariable("x1", 0, None, pulp.LpContinuous)
  x2 = pulp.LpVariable("x2", 0, None, pulp.LpContinuous)
  x3 = pulp.LpVariable("x3", 0, None, pulp.LpContinuous)


  model += (10+a)*x1+16*x2+8*x3, "Revenue"

  model += 2*x1+2*x2 <= 10, "Limit on 2x2"
  model += x1+2*x2+x3 <= 8, "Limit on 2x4"
  model += x3 <= 8, "Limit on 1x2"

  model.solve()
  o = {'delta':a, 'x1':x1.varValue, 'x2':x2.varValue, 'x3':x3.varValue, 'Obj':pulp.value(model.objective)}
  return(o)

In [0]:
output=[]
for i in np.arange(-5,5,0.5):
  output.append(run_pulp_model(i))
df = pd.DataFrame(output)

plt.scatter(df['delta'],df['x1'])
plt.scatter(df['delta'],df['x2'])
plt.scatter(df['delta'],df['x3'])
plt.grid()
plt.show()

In [0]:
df[5:10]

## Sensitivity Analysis Report

The following code installs the GLPK solver (when running this notebook in Colab) and generates its sensitivity analysis report.

In [0]:
!pip install pulp
!sudo apt-get install glpk-utils

import pulp

model = pulp.LpProblem("Lego_Construction_Problem", pulp.LpMaximize)
x1 = pulp.LpVariable("x1", 0, None, pulp.LpContinuous)
x2 = pulp.LpVariable("x2", 0, None, pulp.LpContinuous)
x3 = pulp.LpVariable("x3", 0, None, pulp.LpContinuous)


model += (10+a)*x1+16*x2+8*x3, "Revenue"

model += 2*x1+2*x2 <= 10, "Limit on 2x2"
model += x1+2*x2+x3 <= 8, "Limit on 2x4"
model += x3 <= 8, "Limit on 1x2"

model.solve(pulp.GLPK_CMD(options=['--ranges', 'sensit.txt']))

from google.colab import files

files.download('sensit.txt')